In [1]:
import cv2
import imutils
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from deskew import determine_skew
import face_recognition
import time
from skimage.transform import rotate
from deskew import determine_skew

In [2]:
def face_detect_rotate(image):
    trials = 0
    while(trials<4):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        face_locations = face_recognition.face_locations(gray)
        if(len(face_locations)>0):
            main_face = max(face_locations,key = lambda x : abs(x[0] - x[2]) * abs(x[1] - x[3]))
            leftX = main_face[1]
            return (leftX,image)
        else : 
            image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE) 
            trials += 1
    return 0
def traditional_processing(image,kw,kh):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kw, kh))
    gray = cv2.GaussianBlur(gray, (3, 3), 0)
    gray = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)
    gray = cv2.Sobel(gray, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
    grad = np.absolute(gray)
    (minVal, maxVal) = (np.min(grad), np.max(grad))
    grad = (grad - minVal) / (maxVal - minVal)
    grad = (grad * 255).astype("uint8")
    grad = cv2.morphologyEx(grad, cv2.MORPH_CLOSE, kernel)
    thresh = cv2.threshold(grad, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    thresh = cv2.dilate(thresh, None, iterations=3)
    return thresh
def show(image):
    fig = plt.figure(figsize=(10, 10))
    rows = 1
    columns = 1
    fig.add_subplot(rows, columns, 1)
    plt.imshow(image)
    plt.title("image")
    plt.axis('off')
def show2(im1,im2):
    fig = plt.figure(figsize=(15, 10))
    rows = 1
    columns = 2
    fig.add_subplot(rows, columns, 1)
    plt.imshow(im1)
    plt.title("image1")
    plt.axis('off')
    fig.add_subplot(rows, columns, 2)
    plt.imshow(im2)
    plt.title("image1")
    plt.axis('off')

In [3]:
from helper_functions import *

In [4]:
relative_path = "../Docs/dataset"
passport_list = os.listdir(relative_path)
passport_list = [p for p in passport_list if p[-4:]==".jpg"]

In [5]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
from easyocr import Reader
from difflib import SequenceMatcher

In [6]:
def init():
    readerLatin = Reader(["en"], gpu=True)
    readerArabic = Reader(["ar"], gpu=True)
    readermix = Reader(["ar","en"], gpu=True)
    return (readerLatin,readerArabic,readermix )
(readerLatin,readerArabic,readermix ) = init()

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [7]:
dataset = "datasets/script_dir"
script = os.path.join(dataset,"script")
noscript = os.path.join(dataset,"noscript")

In [14]:
def extract_clips(image,kw,kh,grp_id):
    original = image.copy()
    image = traditional_processing(image,kw,kh)
    im_h,im_w = image.shape[:2]
    cnts = cv2.findContours(image.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    exp_id = 0
    for c in cnts:
        (x, y, w, h) = cv2.boundingRect(c)
        (xstart,ystart,xend,yend)=(x,y,x+w,y+h)
        clip = original[ystart:yend,xstart:xend]
        kernel = np.array([[0, -1, 0],[-1, 5,-1],[0, -1, 0]])
        clip = cv2.filter2D(src=clip, ddepth=-1, kernel=kernel)
        textsmix = [res[1] for res in readermix.readtext(clip)]
        textsara = [res[1] for res in readerArabic.readtext(clip)]
        textsltn = [res[1] for res in readerLatin.readtext(clip)]
        texts = textsmix + textsara + textsltn
        text = " ".join(texts)
        if(text==""):
            cv2.imwrite(noscript+"/"+str(grp_id)+str(exp_id)+".jpg",clip)
        else:
            cv2.imwrite(script+"/"+str(grp_id)+str(exp_id)+".jpg",clip)
        exp_id+=1

In [15]:
def deskewing(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    angle = determine_skew(gray)
    rotated = rotate(image, angle, resize=True) 
    return rotated

In [16]:
grp_id = 0
for passport in passport_list:
    image = cv2.imread(os.path.join(relative_path,passport))
    original = image.copy()
    res = face_detect_rotate(image)
    try:
        _,rotated = res
        image = rotated
        kernel = np.array([[0, -1, 0],[-1, 5,-1],[0, -1, 0]])
        image = cv2.filter2D(src=image, ddepth=-1, kernel=kernel)
        (h,w) = image.shape[:2]
        area = (h*w)
        #image = deskewing(image)
        while(area < 900*1500):
            image = cv2.resize(image, ( int(w*1.4), int(h*1.4) ), interpolation = cv2.INTER_AREA)
            (h,w) = image.shape[:2]
            area = (h*w)
        extract_clips(image,40,8,grp_id)
    except Exception as e: 
        print("/ skipping nb : ",grp_id,"  face not found /",e )
    grp_id += 1
    print( str( int (grp_id / len(passport_list) * 100) )+ "%" ,end="=> ")
print("completed")

7%=> 15%=> 23%=> 30%=> 38%=> 46%=> 53%=> 61%=> 69%=> 76%=> 84%=> 92%=> 100%=> completed


In [11]:
image = cv2.imread("testing.jpg")
image.shape

(462, 720, 3)

In [31]:
text = " ".join("")
print(len(text))

0


In [13]:
print(len(os.listdir(script)))
print(len(os.listdir(noscript)))

714
1701


In [12]:
for i in os.listdir(noscript) : 
    path = os.path.join(noscript,i)
    im = cv2.imread(path)
    (h,w) = im.shape[:2]
    if(h*w <=100):
        os.remove(path)

In [66]:
classes = [script,noscript]
rows = []
for c in classes:
    records = os.listdir(c)
    i = classes.index(c)
    for r in records:
        path = path = os.path.join(c,r)
        im = cv2.imread(path)
        (h,w) = im.shape[:2]
        area = h*w
        ratio = h/w
        row = [h,w,area,ratio,i]
        rows.append(row)
df = pd.DataFrame(rows)
df

,0,1,2,3,4
0,47,1182,55554,0.039763,0
1,48,1182,56736,0.040609,0
2,34,222,7548,0.153153,0
3,44,209,9196,0.210526,0
4,37,302,11174,0.122517,0
...,...,...,...,...,...
2372,9,39,351,0.230769,1
2373,26,90,2340,0.288889,1
2374,25,37,925,0.675676,1
2375,29,72,2088,0.402778,1


In [67]:
df.describe()

,0,1,2,3,4
count,2377.000000,2377.000000,2.377000e+03,2377.000000,2377.000000
mean,44.911653,140.961296,1.142842e+04,0.987088,0.699621
std,87.001479,229.701498,4.946799e+04,2.018593,0.458519
min,7.000000,7.000000,1.040000e+02,0.026477,0.000000
25%,15.000000,17.000000,2.790000e+02,0.237500,0.000000
50%,28.000000,48.000000,1.372000e+03,0.490000,1.000000
75%,47.000000,166.000000,7.455000e+03,1.050000,1.000000
max,2000.000000,1747.000000,1.806336e+06,47.119048,1.000000


In [68]:
df[4==0].describe()

KeyError: False

In [17]:
classes = [script,noscript]
for c in classes:
    records = os.listdir(c)
    for r in records:
        path = path = os.path.join(c,r)
        im = cv2.imread(path)
        resized = cv2.resize(im, (228,228), interpolation = cv2.INTER_AREA)
        os.remove(path)
        cv2.imwrite(path,resized)